# Set Up
- Download the ECG dataset hosted on kaggle. **This step requires a Kaggle API token.**
- Clone the project repository to access the experiment models

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

# make sure there is a kaggle.json file
!ls -lha kaggle.json

# install the Kaggle API token
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (4).json
-rw-r--r-- 1 root root 67 Nov 11 00:26 kaggle.json


In [3]:
# download and unzip the ECG dataset hosted on kaggle
!kaggle datasets download -d shayanfazeli/heartbeat
!unzip -q heartbeat.zip

# clone the project github repository
!git clone https://github.com/distributedgarden/tsc_attention.git

heartbeat.zip: Skipping, found more recently modified local copy (use --force to force download)
replace mitbih_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace mitbih_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ptbdb_abnormal.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ptbdb_normal.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
fatal: destination path 'tsc_attention' already exists and is not an empty directory.


# LSTM Experiment
### Description:
Use the ECG dataset to train the basic LSTM model and evaluate its performance.


### Steps:
1. split the ECG data into train and test subsets
1. preprocess the subsets (standard scaling)
1. convert to pytorch tensors
1. set up the LSTM model
1. train the LSTM model
1. evaluate performance with accuracy, F1 score, precision, recall

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

%cd tsc_attention
from src.models.lstm import LSTM
from src.models.attention_lstm import AttentionLSTM
%cd ..


/content/tsc_attention
/content


In [5]:
# load data
train_df = pd.read_csv("mitbih_train.csv", header=None)
test_df = pd.read_csv("mitbih_test.csv", header=None)

# last column is the label
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

print(len(train_df))
print(len(test_df))
print(len(X_train[1]))
print(X_train[1])

87554
21892
187
[0.96011394 0.86324787 0.46153846 0.1965812  0.0940171  0.12535612
 0.0997151  0.08831909 0.07407407 0.08262108 0.07407407 0.06267806
 0.06552707 0.06552707 0.06267806 0.07692308 0.07122507 0.08262108
 0.09116809 0.09686609 0.08262108 0.08262108 0.09116809 0.10541311
 0.12250713 0.14814815 0.18233618 0.19373219 0.21367522 0.20797721
 0.22222222 0.25356126 0.27065527 0.28774929 0.28490028 0.29344729
 0.25641027 0.24786325 0.18803419 0.14529915 0.10826211 0.08262108
 0.07977208 0.07407407 0.01424501 0.01139601 0.06267806 0.05128205
 0.05698006 0.04843305 0.02849003 0.03133903 0.07692308 0.02564103
 0.02849003 0.03703704 0.0940171  0.08547009 0.03988604 0.05982906
 0.07407407 0.07977208 0.09116809 0.0997151  0.10826211 0.08831909
 0.09116809 0.06552707 0.08547009 0.08831909 0.07692308 0.08262108
 0.09686609 0.0997151  0.13390313 0.1025641  0.03988604 0.06552707
 0.07407407 0.08262108 0.08547009 0.05698006 0.04558405 0.1025641
 0.03988604 0.01139601 0.01709402 0.03133903 0.

In [6]:
# normalize the ECG signals
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to tensors
X_train_tensor = torch.tensor(X_train).float().unsqueeze(2)  # Adding channel dimension
y_train_tensor = torch.tensor(y_train).long()
X_test_tensor = torch.tensor(X_test).float().unsqueeze(2)  # Adding channel dimension
y_test_tensor = torch.tensor(y_test).long()

# Dataset objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
#%cd tsc_attention

# set parameters
# ECG data is univariate, so the input dimension is 1
input_dim = 1
hidden_dim = 128
num_layers = 2
num_classes = 5

model = LSTM(input_dim, hidden_dim, num_layers, num_classes)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

LSTM(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)


In [8]:
# train
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# evaluate
model.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.append(labels)
        predicted_labels.append(predicted)

# combine lists of results
true_labels = torch.cat(true_labels).cpu().numpy()
predicted_labels = torch.cat(predicted_labels).cpu().numpy()

# metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average="weighted")
recall = recall_score(true_labels, predicted_labels, average="weighted")
f1 = f1_score(true_labels, predicted_labels, average="weighted")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch [1/10], Step [100/2737], Loss: 0.9022
Epoch [1/10], Step [200/2737], Loss: 0.9652
Epoch [1/10], Step [300/2737], Loss: 0.3407
Epoch [1/10], Step [400/2737], Loss: 0.8190
Epoch [1/10], Step [500/2737], Loss: 0.6633
Epoch [1/10], Step [600/2737], Loss: 0.6305
Epoch [1/10], Step [700/2737], Loss: 0.7176
Epoch [1/10], Step [800/2737], Loss: 0.4364
Epoch [1/10], Step [900/2737], Loss: 0.1948
Epoch [1/10], Step [1000/2737], Loss: 0.5222
Epoch [1/10], Step [1100/2737], Loss: 0.7874
Epoch [1/10], Step [1200/2737], Loss: 0.3346
Epoch [1/10], Step [1300/2737], Loss: 0.6266
Epoch [1/10], Step [1400/2737], Loss: 0.4441
Epoch [1/10], Step [1500/2737], Loss: 0.2740
Epoch [1/10], Step [1600/2737], Loss: 0.6414
Epoch [1/10], Step [1700/2737], Loss: 0.3938
Epoch [1/10], Step [1800/2737], Loss: 0.7288
Epoch [1/10], Step [1900/2737], Loss: 0.6784
Epoch [1/10], Step [2000/2737], Loss: 0.4890
Epoch [1/10], Step [2100/2737], Loss: 0.7913
Epoch [1/10], Step [2200/2737], Loss: 0.4953
Epoch [1/10], Step 

# LSTM with Attention Experiment
### Description
Use the ECG dataset to train the LSTM+Attention model and evaluate its performance.

### Steps
1. set up the LSTM+Attention model
1. train the LSTM+Attention model
1. evaluate performance with accuracy, F1 score, precision, recall

In [9]:
#%cd tsc_attention

# set parameters
input_dim = 1
hidden_dim = 128
num_layers = 2
num_classes = 5

attention_model = AttentionLSTM(input_dim, hidden_dim, num_layers, num_classes)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(attention_model.parameters(), lr=0.001)

print(attention_model)

AttentionLSTM(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True)
  (attention): SelfAttention(
    (query): Linear(in_features=128, out_features=128, bias=True)
    (key): Linear(in_features=128, out_features=128, bias=True)
    (value): Linear(in_features=128, out_features=128, bias=True)
  )
  (fc): Linear(in_features=128, out_features=5, bias=True)
)


In [10]:
# train
num_epochs = 10
for epoch in range(num_epochs):
    attention_model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = attention_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# evaluate
attention_model.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = attention_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.append(labels)
        predicted_labels.append(predicted)

# combine lists of results
true_labels = torch.cat(true_labels).cpu().numpy()
predicted_labels = torch.cat(predicted_labels).cpu().numpy()

# metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average="weighted")
recall = recall_score(true_labels, predicted_labels, average="weighted")
f1 = f1_score(true_labels, predicted_labels, average="weighted")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch [1/10], Step [100/2737], Loss: 0.9986
Epoch [1/10], Step [200/2737], Loss: 1.1548
Epoch [1/10], Step [300/2737], Loss: 1.0923
Epoch [1/10], Step [400/2737], Loss: 1.1236
Epoch [1/10], Step [500/2737], Loss: 1.1548
Epoch [1/10], Step [600/2737], Loss: 1.0923
Epoch [1/10], Step [700/2737], Loss: 1.1236
Epoch [1/10], Step [800/2737], Loss: 1.0298
Epoch [1/10], Step [900/2737], Loss: 1.1861
Epoch [1/10], Step [1000/2737], Loss: 1.0611
Epoch [1/10], Step [1100/2737], Loss: 0.9361
Epoch [1/10], Step [1200/2737], Loss: 1.0611
Epoch [1/10], Step [1300/2737], Loss: 1.0923
Epoch [1/10], Step [1400/2737], Loss: 0.9361
Epoch [1/10], Step [1500/2737], Loss: 1.0298
Epoch [1/10], Step [1600/2737], Loss: 1.2173
Epoch [1/10], Step [1700/2737], Loss: 1.0923
Epoch [1/10], Step [1800/2737], Loss: 1.0298
Epoch [1/10], Step [1900/2737], Loss: 0.9986
Epoch [1/10], Step [2000/2737], Loss: 0.9986
Epoch [1/10], Step [2100/2737], Loss: 1.1861
Epoch [1/10], Step [2200/2737], Loss: 1.0298
Epoch [1/10], Step 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
